In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


In [4]:
# download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [5]:
# create a dataloader
batch_size = 64

In [6]:
# create dataloaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [8]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"using {device} device")

using mps device


In [10]:
# define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epoch = 5
for t in range(epoch):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("done!")

Epoch 1
-------------------------------
loss: 0.952832  [   64/60000]
loss: 1.002673  [ 6464/60000]
loss: 0.795184  [12864/60000]
loss: 0.991507  [19264/60000]
loss: 0.841486  [25664/60000]
loss: 0.864880  [32064/60000]
loss: 0.938733  [38464/60000]
loss: 0.870486  [44864/60000]
loss: 0.910932  [51264/60000]
loss: 0.862095  [57664/60000]
Test Error: 
 Accuracy: 70.1%, Avg loss: 0.864288 

Epoch 2
-------------------------------
loss: 0.882185  [   64/60000]
loss: 0.948767  [ 6464/60000]
loss: 0.731526  [12864/60000]
loss: 0.941709  [19264/60000]
loss: 0.795782  [25664/60000]
loss: 0.815357  [32064/60000]
loss: 0.894161  [38464/60000]
loss: 0.830904  [44864/60000]
loss: 0.868018  [51264/60000]
loss: 0.823871  [57664/60000]
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.822503 

Epoch 3
-------------------------------
loss: 0.828173  [   64/60000]
loss: 0.907319  [ 6464/60000]
loss: 0.683434  [12864/60000]
loss: 0.904207  [19264/60000]
loss: 0.762094  [25664/60000]
loss: 0.778037  [32064/600

In [19]:
import os
os.makedirs("models", exist_ok=True)

In [20]:
torch.save(model.state_dict(), "models/test_model.pth")
print("saved PyTorch Model State to test_model.pth")

saved PyTorch Model State to test_model.pth
